In [1]:
import numpy as np
import tensorflow as tf
# Import plt from matplotlib
import matplotlib.pyplot as plt
import glob
import random
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split
import json
import os
import datetime
import time
import shutil

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="gravitational-waves", entity="viktor-cikojevic")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: viktor-cikojevic. Use `wandb login --relogin` to force relogin
wandb: ERROR Failed to sample metric: Not Supported


In [3]:
BATCH_SIZE = 8
STEPS_PER_EPOCH = 16
VALIDATION_STEPS = 4
EPOCHS = 100

In [4]:
# Read the data.
# root = "/Volumes/T7/gravitational-waves"
root = "/media/viktor/T7/gravitational-waves"

all_files =  glob.glob(f'{root}/generated-signals/signals_h0_0.1/*.npy') + glob.glob(f'{root}/generated-signals/signals_h0_0.0/*.npy')
print("There are {} files".format(len(all_files)))
# Shuffle all_files
random.shuffle(all_files)
# Train/test split all_files
train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)

# Create labels for train and test data
train_labels = np.array([int(file.split("/")[-2] == 'signals_h0_0.1') for file in train_files])
test_labels = np.array([int(file.split("/")[-2] == 'signals_h0_0.1') for file in test_files])

# Load numpy arrays with tf.data.Dataset. Paths to numpy arrays are given in train_files and test_files. Data has a label 0 if it's a signal from h0, and 1 if it's a signal from h1.
train_dataset = tf.data.Dataset.from_tensor_slices(train_files)
test_dataset = tf.data.Dataset.from_tensor_slices(test_files)
# Load numpy arrays from file paths
train_dataset = train_dataset.map(lambda x: tf.numpy_function(np.load, [x], [tf.float32]))
test_dataset = test_dataset.map(lambda x: tf.numpy_function(np.load, [x], [tf.float32]))


# Create tf.data.Dataset from train_labels and test_labels
train_labels = tf.data.Dataset.from_tensor_slices(train_labels)
test_labels = tf.data.Dataset.from_tensor_slices(test_labels)

# Combine train_dataset and train_labels
train_dataset = tf.data.Dataset.zip((train_dataset, train_labels))
test_dataset = tf.data.Dataset.zip((test_dataset, test_labels))

# Shuffle and batch the data
train_dataset = train_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
# Repeat the data
train_dataset = train_dataset.repeat()
test_dataset = test_dataset.repeat()

tmp_train = next(train_dataset.as_numpy_iterator())
tmp_test = next(test_dataset.as_numpy_iterator())
# tmp_train[0][0].shape
tmp_train[0][0].shape, tmp_train[1]

There are 4000 files


2022-11-05 10:59:29.248134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-05 10:59:29.279430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-05 10:59:29.279686: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-05 10:59:29.280200: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

((8, 4, 360, 360), array([1, 0, 0, 1, 0, 0, 1, 1]))

In [5]:
np.sum(["signals_noise" in file  for file in test_files]) / len(test_files), len(test_files), len(train_files)

(0.0, 800, 3200)

In [6]:
# Create a ResNet model. Load from keras.applications.resnet50.ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
# Create a ResNet50 model with weights not pre-trained on  ImageNet. Set include_top to False.  Set input_shape to (360, 360, 4). Set pooling to 'avg'. Classify into 2 classes. Activate sigmoid function.
resnet = ResNet50(weights=None, include_top=False, input_shape=(360, 360, 4), pooling='avg', classes=1)
model = tf.keras.Sequential([
    tf.keras.layers.Permute((2, 3, 1), input_shape=(4, 360, 360)),
    resnet,
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])


model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.01), 
              loss = tf.keras.losses.BinaryFocalCrossentropy(gamma=4),
              metrics=[tf.keras.metrics.AUC()]
              )
model.summary()



/home/viktor/.local/lib/python3.10/site-packages/keras/applications/resnet.py:135: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 360, 360, 4)       0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23590848  
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 23,722,049
Trainable params: 23,668,929
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
# path = "resnet-classifier/signals_true_mix_vs_noise/29-10-2022_22:23:46/best_model.h5"
# model.load_weights(path)

In [8]:
# tf.keras.utils.plot_model(model, show_shapes=True)

In [9]:
model.evaluate(train_dataset.take(4))

2022-11-05 10:59:31.782875: I tensorflow/core/grappler/optimizers/generic_layout_optimizer.cc:353] Cancel Transpose nodes around Pad: transpose_before=sequential/permute/transpose pad=sequential/resnet50/conv1_pad/Pad transpose_after=sequential/resnet50/conv1_conv/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer
2022-11-05 10:59:32.349900: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204
2022-11-05 10:59:33.609525: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-05 10:59:33.609567: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gain

4/4 [==============================] - 4s 161ms/step - loss: 0.1740 - auc: 0.4615


[0.17404595017433167, 0.4615384638309479]

In [10]:
# Evaluate the model on the test dataset. 
# model(tmp_test[1]).shape
model.evaluate(test_dataset.take(4))

4/4 [==============================] - 1s 165ms/step - loss: 0.1720 - auc: 0.5000


[0.17202134430408478, 0.5]

In [11]:
# Train the model, and evaluate it on the test dataset. steps_per_epoch=4, epochs=10, validation_data=test_dataset, validation steps=4. Save the best model during training to 'best_model.h5'
date = datetime.datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

history = model.fit(train_dataset, 
          steps_per_epoch=50, 
          epochs=20, 
          validation_data=test_dataset, 
          validation_steps=10,
          callbacks=[WandbCallback()]
          )

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/50


2022-11-05 10:59:38.863513: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-05 10:59:38.863550: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-05 10:59:38.881566: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-05 10:59:38.881584: W tensorflow/core/common_runtime/bfc_allocato

50/50 [==============================] - ETA: 0s - loss: 0.1900 - auc: 0.4611

INFO:tensorflow:Assets written to: /media/viktor/T7/gravitational-waves/notebooks/2-train-resnet-models/wandb/run-20221105_105926-2ryuicn4/files/model-best/assets


...

50/50 [==============================] - 40s 812ms/step - loss: 0.0667 - auc: 0.9899 - val_loss: 0.1774 - val_auc: 0.4849
Epoch 15/50
50/50 [==============================] - ETA: 0s - loss: 0.0567 - auc: 0.9944

INFO:tensorflow:Assets written to: /media/viktor/T7/gravitational-waves/notebooks/2-train-resnet-models/wandb/run-20221105_105926-2ryuicn4/files/model-best/assets


INFO:tensorflow:Assets written to: /media/viktor/T7/gravitational-waves/notebooks/2-train-resnet-models/wandb/run-20221105_105926-2ryuicn4/files/model-best/assets
wandb: Adding directory to artifact (/media/viktor/T7/gravitational-waves/notebooks/2-train-resnet-models/wandb/run-20221105_105926-2ryuicn4/files/model-best)... Done. 0.6s


50/50 [==============================] - 39s 780ms/step - loss: 0.0567 - auc: 0.9944 - val_loss: 0.1652 - val_auc: 0.5797
Epoch 16/50
38/50 [=====================>........] - ETA: 5s - loss: 0.0490 - auc: 0.9967

KeyboardInterrupt: 